In [1]:
from colors import ColorsCorpusReader
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch_color_describer import (
    ContextualColorDescriber, create_example_dataset)
import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL

In [2]:
utils.fix_random_seeds()

In [3]:
COLORS_SRC_FILENAME = os.path.join(
    "data", "colors", "filteredCorpus.csv")

In [4]:
corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME,
    word_count=None,
    normalize_colors=True)

In [17]:
# Dev dataset
dev_color_seqs, dev_word_seqs, dev_vocab = create_example_dataset(
    group_size=100, vec_dim=2)
dev_color_seqs_train, dev_color_seqs_test, dev_word_seqs_train, dev_word_seqs_test = \
    train_test_split(dev_color_seqs, dev_word_seqs)

In [6]:
# Debug dataset
toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
    group_size=20, vec_dim=2)
toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
    train_test_split(toy_color_seqs, toy_word_seqs)


# Baseline Model

In [18]:
import torch.nn as nn
from torch_color_describer import Encoder, Decoder

class DeepEncoder(Encoder):
    def __init__(self, *args, num_layers=2, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_layers = num_layers
        self.rnn = nn.GRU(
            input_size=self.color_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True)


class DeepDecoder(Decoder):
    def __init__(self, *args, num_layers=2, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_layers = num_layers
        self.rnn = nn.GRU(
            input_size=self.embed_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True)

In [19]:
from torch_color_describer import EncoderDecoder

class DeepContextualColorDescriber(ContextualColorDescriber):
    def __init__(self, *args, num_layers=2, **kwargs):
        self.num_layers = num_layers
        super().__init__(*args, **kwargs)

    def build_graph(self):
        encoder = DeepEncoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim,
            num_layers=self.num_layers)  # The new piece is this argument.

        decoder = DeepDecoder(
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim,
            num_layers=self.num_layers)  # The new piece is this argument.

        return EncoderDecoder(encoder, decoder)

In [20]:
mod_baseline = DeepContextualColorDescriber(
    dev_vocab,
    embed_dim=10,
    hidden_dim=10)

In [21]:
%time _ = mod_baseline.fit(dev_color_seqs_train, dev_word_seqs_train)

Stopping after epoch 611. Training loss did not improve more than tol=1e-05. Final error is 0.1754165217280388.

CPU times: user 1min 11s, sys: 10.9 s, total: 1min 22s
Wall time: 1min 1s


In [22]:
mod_baseline.listener_accuracy(dev_color_seqs_test, dev_word_seqs_test)

1.0

In [25]:
baseline_eval = mod_baseline.evaluate(dev_color_seqs_test, dev_word_seqs_test)
baseline_perp = mod_baseline.perplexities(dev_color_seqs_test, dev_word_seqs_test)
print("Listener acc: {0}, BLEU: {1}, Perp: {2}".format(
    baseline_eval['listener_accuracy'], baseline_eval['corpus_bleu'], baseline_perp))

Listener acc: 1.0, BLEU: 1.0, Perp: [1.0023646685310066, 1.0045776168152944, 1.004586382066643, 1.0046199791826271, 1.002347660933698, 1.0045150748508154, 1.0046288850193703, 1.0044780435768654, 1.0023501277659816, 1.0023555428488387, 1.0046123460481107, 1.0023484230418926, 1.0044653661248146, 1.0023478013218756, 1.002350368433846, 1.0044664784173167, 1.002359995337962, 1.0046172739024042, 1.0045992762739389, 1.0046263301778775, 1.0023713074229, 1.0023518726132354, 1.0023548408870522, 1.0023747573092807, 1.004595501238899, 1.0023524943433537, 1.002364648474198, 1.0023486637081198, 1.0045050162987212, 1.0045666378131222, 1.004595008850345, 1.004657473616565, 1.0045957166594226, 1.0046176707321879, 1.0023507695474672, 1.0044715219030558, 1.0023597145480225, 1.0024246656298978, 1.00461394651042, 1.0023584710535005, 1.0023587919547219, 1.0023549211111569, 1.0047136972072794, 1.004700911270965, 1.004700295333324, 1.0044879941086238, 1.0046676441570044, 1.0045431042914095, 1.0023535372489514

In [30]:
"""
print(mod_baseline.params)

['batch_size', 'max_iter', 'eta', 'optimizer_class', 'l2_strength', 
  'gradient_accumulation_steps', 'max_grad_norm', 'validation_fraction', 
  'early_stopping', 'n_iter_no_change', 'warm_start', 'tol', 'hidden_dim',
  'embed_dim', 'embedding', 'freeze_embedding']
"""
best_baseline_mod = utils.fit_classifier_with_hyperparameter_search(
    dev_color_seqs_train,
    dev_word_seqs_train,
    mod_baseline,
    cv=2,
    scoring=None,
    param_grid={'hidden_dim': [10, 20], 'embed_dim' : [5, 10], 'eta' : [0.001, 0.01, 0.05]})

/Applications/miniconda3/envs/nlu/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
Finished epoch 1 of 1000; error is 3.3078125715255737

['batch_size', 'max_iter', 'eta', 'optimizer_class', 'l2_strength', 'gradient_accumulation_steps', 'max_grad_norm', 'validation_fraction', 'early_stopping', 'n_iter_no_change', 'warm_start', 'tol', 'hidden_dim', 'embed_dim', 'embedding', 'freeze_embedding']


Stopping after epoch 361. Training loss did not improve more than tol=1e-05. Final error is 0.30913403630256653...

Best params: {'embed_dim': 5, 'eta': 0.001, 'hidden_dim': 10}
Best score: 1.000
